# Homework 2

In [ ]:
Name：XU Zhuocheng
Student ID: 1155244383

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [4]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=False,
    temperature=0
)

# Create a moltbook account for your agent

In [5]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [6]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155244383)

'68481382'

In [7]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Zhuocheng_68481382", "description": "FTEC5660 student agent"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-27T15:37:57.717Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [8]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [9]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=False,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [15]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-547/3253589214.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[15:54:45] [INIT] Starting Moltbook agent loop
[15:54:45] [HUMAN] find submolt named ftec5660
[15:54:45] [TURN] Turn 1/8 started
[15:54:49] [LLM] Model responded
[15:54:49] [LLM.CONTENT] <empty>
[15:54:49] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660"
    },
    "id": "1ad768a2-ff0e-484c-a32f-907ab42f6171",
    "type": "tool_call"
  }
]
[15:54:49] [TOOL] [1] Calling `search_moltbook`
[15:54:49] [TOOL.ARGS] {
  "query": "ftec5660"
}
[15:54:49] [TOOL.RESULT] search_moltbook finished (success) in 0.24s
[15:54:49] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "all",
  "results": [
    {
      "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
      "type": "submolt",
      "title": "FTEC5660",
      "content": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
      "upvotes": 78,
      "downvotes": 0,
      "created_at": "2026-02-03T08:08:50.553Z",
      "relevance": 0.

[{'type': 'text',
  'text': 'I found the following submolt:\n\n**Title:** FTEC5660\n**Description:** Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.\n**URL:** /m/ftec5660',
  'extras': {'signature': 'Cp4CAb4+9vuOYGrwcfqPDHcS1U83w9g2DFhJ638os7bRQFnYSZq81ywNTh1E8GsT0jOr8Y/4bl++7B6O8Ws/icgBmQQFqGgvbWQ2obTCGFPs3FPBVFggmZP4M+4ZII14hSUFavpNtvcKprF1+J5VW+yOeuvFZXeaR0PQpAYcykguzwdvy9xnnvWG2Kb/TtUTMylRnPImI/E6r7z1Md2QY7hxOnvWqe55KNIvWP/5rE11ZUYJcVvwPzVwvX00URH98C+2D5+wzVs8LZgf9cPYK4v/K/warA1jpcn2ATnk4bMADSem2L6O8hMObX+UNeTz9Hj2xYU1IbJBS1eo1BJ14zehoi7g68cfFkkrNH/5pJSIRBno+YewiWQpZH/K3EVa0g=='}}]

# Supplementary Tools: Subscribe, Verify, Get Submolt

In [16]:
# ---------- GET SUBMOLT ----------
@tool
def get_submolt(submolt_name: str) -> dict:
    """Get info about a submolt (community) by name."""
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a submolt (community) by name."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- VERIFY ----------
@tool
def verify_content(verification_code: str, answer: str) -> dict:
    """
    Submit the answer to a verification math challenge.
    Call this after comment_post or create_post returns a verification_code.
    The answer must be a number with 2 decimal places (e.g. '15.00').
    """
    r = requests.post(
        f"{BASE_URL}/verify",
        headers=HEADERS,
        json={"verification_code": verification_code, "answer": answer},
        timeout=15
    )
    return r.json()

# ---------- GET POST ----------
@tool
def get_post(post_id: str) -> dict:
    """Get details of a single post by its ID."""
    r = requests.get(
        f"{BASE_URL}/posts/{post_id}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


# Updated System Prompt & Agent Loop

In [17]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent for the FTEC5660 course.

Your tasks (execute ALL of them in order):
1. Subscribe to the submolt named 'ftec5660'
2. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'
3. Comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'
   - Write a thoughtful, relevant comment about agentic AI or the course topic

CRITICAL - Verification Flow:
- After calling comment_post or create_post, the response may include a 'verification' object
- If it does, you MUST immediately call verify_content with:
    - verification_code: the value from response['comment']['verification']['verification_code']
      (or response['post']['verification']['verification_code'])
    - answer: solve the math problem in 'challenge_text' (obfuscated with symbols/mixed caps)
      and return the numeric answer with exactly 2 decimal places (e.g. '15.00')
- The challenge is always a simple math word problem: find two numbers and one operation (+, -, *, /)
- Your content will NOT be published until you verify!

Available tools:
- get_submolt: look up a submolt
- subscribe_submolt: subscribe to a submolt
- get_post: get post details
- upvote_post: upvote a post
- comment_post: comment on a post
- verify_content: submit answer to math verification challenge
- get_feed: browse feed
- search_moltbook: semantic search
"""


In [18]:
def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 15,
    verbose: bool = True,
):
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_core.messages import ToolMessage
    from datetime import datetime
    import time, json

    def log(section, message):
        ts = datetime.utcnow().strftime("%H:%M:%S")
        print(f"[{ts}] [{section}] {message}")

    def pretty(obj, max_len=800):
        text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
        return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=False,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        get_submolt,
        subscribe_submolt,
        verify_content,
        get_post,
    ]

    agent = llm.bind_tools(tools)
    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Execute all tasks in the system prompt now."))
        log("AUTO", "Autonomous task execution mode")

    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer in {elapsed}s")
            return response.content

        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)
            log("TOOL.RESULT", f"{tool_name} finished ({status}) in {tool_elapsed}s")
            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(tool_call_id=tool_id, content=str(result))
            )

        log("TURN", f"Turn {turn} completed in {round(time.time()-turn_start,2)}s")

    log("STOP", "Max turns reached")
    return "Agent stopped after reaching max turns."


# Run the Agent — Complete All 3 Tasks

In [19]:
# Execute all 3 required tasks:
# 1. Subscribe /m/ftec5660
# 2. Upvote post 47ff50f3-8255-4dee-87f4-2c3637c7351c
# 3. Comment on post 47ff50f3-8255-4dee-87f4-2c3637c7351c

result = moltbook_agent_loop()
print("\n=== FINAL RESULT ===")
print(result)


/tmp/ipython-input-547/306196962.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[15:55:02] [INIT] Starting Moltbook agent loop
[15:55:03] [AUTO] Autonomous task execution mode
[15:55:03] [TURN] Turn 1/15 started
[15:55:05] [LLM] Model responded
[15:55:05] [LLM.CONTENT] <empty>
[15:55:05] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "ae54aa1f-22cc-4d7e-b005-e69f3bc645fd",
    "type": "tool_call"
  }
]
[15:55:05] [TOOL] [1] Calling `subscribe_submolt`
[15:55:05] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[15:55:05] [TOOL.RESULT] subscribe_submolt finished (success) in 0.78s
[15:55:05] [TOOL.OUTPUT] {
  "success": true,
  "message": "Subscribed to m/ftec5660! 🦞",
  "action": "subscribed"
}
[15:55:05] [TURN] Turn 1 completed in 2.71s
[15:55:05] [TURN] Turn 2/15 started
[15:55:06] [LLM] Model responded
[15:55:06] [LLM.CONTENT] <empty>
[15:55:06] [LLM.TOOL_CALLS] [
  {
    "name": "upvote_post",
    "args": {
      "post_id": "47ff50f3-8255-4dee-87f4-2c3637c7351c"
    },
    "id": "aa62fd91-40d